# Assignment 4: Decision Trees and Ensemble Methods
## Daisy Pinaroc

## 1) Feature Engineering

Let's load the data into a Pandas DataFrame. 

In [107]:
# Load the dataset as a dataframe using pandas library 
import pandas as pd
import numpy as np
# Importing OneHotEncoder class from scikit-learn preprocessing for later use
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

mt_rainier_df = pd.read_csv("MtRainier_data.csv")

# Print a few columns using head() function
mt_rainier_df.head()

,Unnamed: 0,Date,Route,Succeeded,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,0,11/27/2015,Disappointment Cleaver,0,13.643750,26.321667,19.715000,27.839583,68.004167,88.496250
1,1,11/21/2015,Disappointment Cleaver,0,13.749583,31.300000,21.690708,2.245833,117.549667,93.660417
2,2,10/15/2015,Disappointment Cleaver,0,13.461250,46.447917,27.211250,17.163625,259.121375,138.387000
3,3,10/13/2015,Little Tahoma,0,13.532083,40.979583,28.335708,19.591167,279.779167,176.382667
4,4,10/9/2015,Disappointment Cleaver,0,13.216250,38.260417,74.329167,65.138333,264.687500,27.791292


### Exploratory data analysis
Check and remove duplicates. Check and clean rows with NULL entries.

In [108]:
# Dropping duplicated data, if any
mt_rainier_df = mt_rainier_df.drop_duplicates()

# Applying dropna() for completeness
mt_rainier_df = mt_rainier_df.dropna()

# Check the data types of the columns using info() function, also checking for completeness
mt_rainier_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1895 entries, 0 to 1894
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1895 non-null   int64  
 1   Date                   1895 non-null   object 
 2   Route                  1895 non-null   object 
 3   Succeeded              1895 non-null   int64  
 4   Battery Voltage AVG    1895 non-null   float64
 5   Temperature AVG        1895 non-null   float64
 6   Relative Humidity AVG  1895 non-null   float64
 7   Wind Speed Daily AVG   1895 non-null   float64
 8   Wind Direction AVG     1895 non-null   float64
 9   Solare Radiation AVG   1895 non-null   float64
dtypes: float64(6), int64(2), object(2)
memory usage: 162.9+ KB


In [109]:
# Printing the number of rows and columns in the dataframe

# Rows
rows = len(mt_rainier_df.index)

# Columns
col = len(mt_rainier_df.axes[1])

print('Rows:',rows)
print('Columns:', col)

Rows: 1895
Columns: 10


I've determined the number of rows and columns in the dataframe because I would like to see if this changes after feature engineering.

### Feature and Label Definition

In [110]:
mt_rainier_features_df = mt_rainier_df[["Temperature AVG", "Relative Humidity AVG", "Wind Speed Daily AVG", "Wind Direction AVG","Solare Radiation AVG"]] 
mt_rainier_features_df.head()

,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,26.321667,19.715000,27.839583,68.004167,88.496250
1,31.300000,21.690708,2.245833,117.549667,93.660417
2,46.447917,27.211250,17.163625,259.121375,138.387000
3,40.979583,28.335708,19.591167,279.779167,176.382667
4,38.260417,74.329167,65.138333,264.687500,27.791292


In [111]:
mt_rainier_labels_df = mt_rainier_df[["Succeeded"]]
mt_rainier_labels_df.head()

,Succeeded
0,0
1,0
2,0
3,0
4,0


Features to keep:
* Temperature AVG
* Relative Humidity AVG
* Wind Speed Daily AVG
* Wind Direction AVG
* Solare Radiation AVG


* I'm not keeping "Date" because it caused problems with scaling later on. 
* "Battery Voltage AVG" also seems like a feature that won't help the user decide whether to hike on a given day. 

I originally had "Route" as a feature but decided to remove it since we cannot ablate categorical features.

**Succeeded** is our label: carries a binary value for chances of sumitting (1- for success and 0- for no sucess)

### Feature Scaling

Let's scale features using MinMax scaling.

In [112]:
# Creating the scaler 
scaler = MinMaxScaler()

# Scaleing the numerical features with MinMax scaler
numerical_feature_names = ['Temperature AVG', 
                   'Relative Humidity AVG', 
                   'Wind Speed Daily AVG', 
                   'Wind Direction AVG', 
                   'Solare Radiation AVG']
mt_rainier_features_df[numerical_feature_names] = scaler.fit_transform(mt_rainier_features_df[numerical_feature_names])

# Printing out the head of the feature-engineered dataframe
mt_rainier_features_df.head()

C:\Users\daisy\AppData\Local\Temp\ipykernel_16300\2821389350.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mt_rainier_features_df[numerical_feature_names] = scaler.fit_transform(mt_rainier_features_df[numerical_feature_names])


,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,0.395119,0.083886,0.427392,0.204255,0.240442
1,0.496061,0.106431,0.034478,0.389892,0.254473
2,0.803203,0.169424,0.263495,0.920335,0.375994
3,0.692326,0.182255,0.300762,0.997736,0.479228
4,0.637191,0.707076,1.000000,0.941191,0.075508


## 2) Data Splitting

We form the train-test split. The train split can be used for cross validation

In [113]:
# Putting the features and labels into a numpy array for sklearn's data splitter
features = mt_rainier_features_df.to_numpy()
labels = mt_rainier_labels_df.to_numpy()

# Splitting the data into test and training data
_x, x_test, _y, y_test = train_test_split(features, labels, test_size=0.10, random_state=512)

k = 5 # 5-fold

## 3) Define classifiers and performing cross validation
Let's instantiate our classifiers, namely, 
- Logistic Regression (default)
- Support Vector Classifier (default)
- Decision Tree
- Random Forest
- GradientBoostingClassifier

We will cross validate with k=5 throughout. 

In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


lr_vanilla = LogisticRegression(penalty= "none") # OR penalty="none" depending on your sklearn version
svm_linear = SVC(kernel="linear")
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(random_state=23) # some random seed for reproducibility
grad_boost = GradientBoostingClassifier()

all_models = {"lr":lr_vanilla, 
              "svm":svm_linear,
              "decision_tree":dt,
              "random_forest":rf,
              "grad_boost":grad_boost}

print (f"We are working with classifiers {all_models.keys()}")

We are working with classifiers dict_keys(['lr', 'svm', 'decision_tree', 'random_forest', 'grad_boost'])


In [115]:
# We can use sklearn's cross validation score directly
# We can speed up training using n_jobs parameter which specifies how many cpu_cores to use

best_model_name = ""
best_model_valid_accuracy = 0
best_model = None

for model_name in all_models.keys():
    model = all_models[model_name]
    cv_scores = cross_val_score(model,_x,_y.flatten(), cv=k, n_jobs=4) 
    average_cv_score = cv_scores.mean()
    print (f"Mean cross validation accuracy for model {model_name} = {average_cv_score}")

    if average_cv_score > best_model_valid_accuracy :
        best_model_name = model_name
        best_model_valid_accuracy  = average_cv_score
        best_model = model

print (f"Best model is {best_model_name} with {k}-fold accuracy of {best_model_valid_accuracy}")

Mean cross validation accuracy for model lr = 0.6017595307917889
Mean cross validation accuracy for model svm = 0.5929618768328446
Mean cross validation accuracy for model decision_tree = 0.6093841642228739
Mean cross validation accuracy for model random_forest = 0.6052785923753665
Mean cross validation accuracy for model grad_boost = 0.6170087976539589
Best model is grad_boost with 5-fold accuracy of 0.6170087976539589


### Testing Accuracy
Let's print the testing accuracy

In [119]:
from sklearn.metrics import accuracy_score

# Let's fit the best model again with train+valid data
# This is the model we are gonna ship/deploy to practice

best_model.fit(_x,_y.flatten())

y_pred_test = best_model.predict(x_test)
test_accuracy = accuracy_score(y_pred_test, y_test.flatten())

print(best_model, end='\n\n')
print (f"Test accuracy for model {test_accuracy}")

GradientBoostingClassifier()

Test accuracy for model 0.6526315789473685


### Test Accuracy for best classifier
0.653: Gradient Boosting Classifier

## 4) Feature Ranking - Ablation Test
To assess feature importance and rank features based on their importance, we will perform ablation test models.

We skip one feature at a time and run our k-fold validation for a model of our choice to examine how much impact does droping that feature have on the overall accuracy

In [117]:
# Let's run ablation tests on our best model
# You could choose any model to do this test
best_model = GradientBoostingClassifier()

feature_names = mt_rainier_features_df.columns

# Let's maintain an accuracy dictionary

accuracy_drop_log = {"No ablation":0}

for i in range(len(feature_names)):
    # we are going to drop one feature at a time
    feature_name = feature_names[i]
    print (f"Removing feature {feature_name}")
    
    # Remmeber? We have the train + valid data in the above section?
    # We just remove the feature by not selecting the column from the index i

    x_ablated = np.delete(_x,i,axis=1) # axis = 1 means columns
    
    cv_scores = cross_val_score(best_model,x_ablated,_y.flatten(), cv=k, n_jobs=4)
    average_cv_score = cv_scores.mean()
    print (f"Mean cross validation accuracy = {average_cv_score}")
    accuracy_drop_log[feature_name] = best_model_valid_accuracy-average_cv_score

Removing feature Temperature AVG
Mean cross validation accuracy = 0.612316715542522
Removing feature Relative Humidity AVG
Mean cross validation accuracy = 0.6187683284457478
Removing feature Wind Speed Daily AVG
Mean cross validation accuracy = 0.6129032258064516
Removing feature Wind Direction AVG
Mean cross validation accuracy = 0.6158357771260997
Removing feature Solare Radiation AVG
Mean cross validation accuracy = 0.6076246334310851


### Sorting and printing the features

In [118]:
def criteria(l):
    return l[1]

sorted_accs =  sorted(accuracy_drop_log.items(),key=criteria, reverse=True)

print (f"Features are ranked from best to worst (based on how removing them impacts the accuracy of {best_model_name})")
print (f"**************************************")

i=1
for entry in sorted_accs:
    feature_name = entry[0]
    acc_drop = entry[1]
    
    # We do not want to print "No ablation"
    if feature_name != "No ablation":
        print (f"Feature {i}.{feature_name}, drop in acc {acc_drop}")
        i=i+1


Features are ranked from best to worst (based on how removing them impacts the accuracy of grad_boost)
**************************************
Feature 1.Solare Radiation AVG, drop in acc 0.009384164222873803
Feature 2.Temperature AVG, drop in acc 0.004692082111436902
Feature 3.Wind Speed Daily AVG, drop in acc 0.004105571847507261
Feature 4.Wind Direction AVG, drop in acc 0.0011730205278591699
Feature 5.Relative Humidity AVG, drop in acc -0.0017595307917889214


**Insights:** As seen, factors like Solare Radiation, Temperature AVG, Wind Speed Daily AVG are better predictors than Wind Direction AVG or Relative Humidity based on our dataset and chosen model

* I think it makes sense for Wind Direction and Humidity to have less impact on predicting summit success compared to Solare Radiation, Temperature, or Wind Speed. However, I would think that Wind Speed would have more impact than Solare Radiation at least.